In [ ]:
### THIS PROJECT USES TENSORFLOW=2.15 AND KERAS=3.0.5

In [ ]:
import tensorflow as tf
import cv2
import os
import numpy as np

In [ ]:
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#### Testing

In [ ]:
cap=cv2.VideoCapture(0)
while True:
    ret,frame=cap.read()
    if ret:
        # cv2.rectangle(frame,(140,50),(500,400),(0,0,255),2)
        gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        faces=face_cascade.detectMultiScale(gray,scaleFactor=1.3,minNeighbors=3)
        # a=0,b=0,c=0,d=0
        for (x,y,w,h) in faces:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,255),2)
            # a=x,b=y,c=w,d=h
        cv2.imshow("test",frame[140:500,50:400])
        if cv2.waitKey(1) & 0xFF==ord('q'):
            break
cap.release()
cv2.destroyAllWindows()


In [ ]:
cap.release()

### Data Augmentation

In [ ]:
for img in os.listdir("images"):
    image=cv2.imread(os.path.join("images",img))
    name=img
    new_img=tf.image.stateless_random_flip_up_down(image,seed=(np.random.randint(100),np.random.randint(100)))
    new_img = tf.image.stateless_random_flip_left_right(new_img, seed=(np.random.randint(100),np.random.randint(100)))
    new_img = tf.image.stateless_random_brightness(new_img, max_delta=0.02, seed=(1,2))
    new_img = tf.image.stateless_random_contrast(new_img, lower=0.6, upper=1, seed=(1,3))
    new_img = tf.image.stateless_random_jpeg_quality(new_img, min_jpeg_quality=90, max_jpeg_quality=100, seed=(np.random.randint(100),np.random.randint(100)))
    new_img = tf.image.stateless_random_saturation(new_img, lower=0.9, upper=1, seed=(np.random.randint(100),np.random.randint(100))) 
    cv2.imwrite(os.path.join("images",name),new_img.numpy())

Creating tf data pipeline

In [ ]:
positives=tf.data.Dataset.from_tensor_slices(tf.io.gfile.glob("images/*.jpg")[:5600])

In [ ]:
len(positives)

In [ ]:
negatives=tf.data.Dataset.from_tensor_slices(tf.io.gfile.glob("images/no face/*.jpg")[:4000])

In [ ]:
negatives.as_numpy_iterator().next()

In [ ]:
len(negatives)

In [ ]:
positive=tf.data.Dataset.zip((positives,tf.data.Dataset.from_tensor_slices(tf.ones(len(positives)))))
negative=tf.data.Dataset.zip((negatives,tf.data.Dataset.from_tensor_slices(tf.zeros(len(negatives)))))

In [ ]:
positive,negative.element_spec

In [ ]:
len(positive),len(negative)

Preprocessing image

In [ ]:
def load_image(x):
    byte_image=tf.io.read_file(x)
    img=tf.io.decode_jpeg(byte_image,channels=3)
    img=tf.image.resize(img,(200,200))
    img=img/255.0
    return img

In [ ]:
data=positive.concatenate(negative)

In [ ]:
data

In [ ]:
sample=data.as_numpy_iterator().next()

In [ ]:
sample

In [ ]:
def preprocess(image,label):
    return(load_image(image),label)

In [ ]:
res=preprocess(*sample)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(res[0])

Building data loader pipeline

In [ ]:
data=data.map(preprocess)
data=data.cache()
data=data.shuffle(buffer_size=15000)

In [ ]:
train=data.take(round(len(data)*.7))
train=train.batch(8)
train=train.prefetch(4)

In [ ]:
val=data.skip(round(len(data)*.7))
val=val.take(round(len(data)*.3))
val=val.batch(8)
val=val.prefetch(4)

In [ ]:
len(train),len(val)

In [ ]:
print(train.element_spec,val.element_spec)

### building Feature Extractor model

In [ ]:
from tensorflow.keras.layers import Conv2D,Layer,Input,Dense,MaxPooling2D,Flatten,Concatenate,Normalization
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import L2

In [ ]:
def FeatureExtractor():
    i=Input(shape=(200,200,3))
    c1=Conv2D(64,kernel_size=(5,5),strides=(2,2),activation="relu",name="first_conv")(i)
    m1=MaxPooling2D(pool_size=(2,2),strides=(1,1),padding="same")(c1)
    
    c2=Conv2D(32,kernel_size=(3,3),strides=(2,2),activation="relu",name="second_conv")(m1)
    m2=MaxPooling2D(pool_size=(5,5),strides=(2,2),padding="same")(c2)
    
    c3=Conv2D(128,kernel_size=(5,5),strides=(2,2),activation="relu",name="feature_extractor")(m2)
    
    f=Flatten()(c3)
    
    x=Dense(16,activation='relu',kernel_regularizer=L2())(f)

    return [i,x,c3]

In [ ]:
def EuclideanDistance(feature1,feature2):
    return np.linalg.norm(feature1-feature2)

In [ ]:
## last fully connected dense layer
def addLastLayer():
    i,fully_connected,c3=FeatureExtractor() #input layer, second last fully connected dense layer, feature extractor layer
    x=Dense(1,activation="sigmoid")(fully_connected)
    final_model=Model(inputs=[i],outputs=[x,c3]) #outputs class and features
    final_model.build(input_shape=(200,200,3))
    return final_model

In [ ]:
final_model=addLastLayer()

In [ ]:
final_model.summary()

In [ ]:
loss=tf.keras.losses.binary_crossentropy
accuracy=tf.keras.metrics.BinaryAccuracy(name="accuracy")
opt=tf.keras.optimizers.Adam(learning_rate=3e-6)

creating subclass model

In [ ]:
class Attendance(Model):
    def __init__(self,model,**kwargs):
        super().__init__(**kwargs)
        self.model=model
        
    def compile(self,opt,loss,metric,**kwargs):
        super().compile(**kwargs)
        self.loss=loss
        self.opt=opt
        self.metric=metric
    
    def train_step(self,batch,**kwargs):
        X,y=batch
        y=tf.expand_dims(y, axis=-1)
        
        with tf.GradientTape() as tape:
            y_pred=self.model(X,training=True)
            classLoss=self.loss(y,y_pred[0])
            grad=tape.gradient(classLoss,self.model.trainable_variables)
        opt.apply_gradients(zip(grad,self.model.trainable_variables))
        
        acc=accuracy(y,y_pred[0])
        return {"loss":classLoss,"accuracy":acc}
    
    def test_step(self,batch,**kwargs):
        X,y=batch
        y=tf.expand_dims(y, axis=-1)
        y_pred=self.model(X,training=False)
        vaLoss=self.loss(y,y_pred[0])
        self.metric.update_state(y, y_pred[0])
        return {"loss":vaLoss,"accuracy":self.metric.result()}
    
    def get_config(self):
        base_config = super().get_config()
        config = {
                "submodel": tf.keras.utils.serialize_keras_object(self.model),
                }
        return {**base_config, **config}
    
    @classmethod
    def from_config(cls, config):
        submodel_config = config.pop("submodel")
        submodel = tf.keras.utils.deserialize_keras_object(submodel_config)
        return cls(model=submodel, **config)
    
    def call(self,X,**kwargs):
        return self.model(X,**kwargs)

In [ ]:
model=Attendance(final_model)

In [ ]:
model.compile(opt=opt,loss=loss,metric=accuracy)

In [ ]:
hist=model.fit(train,epochs=5,validation_data=val)

In [ ]:
model.predict(test_img)

In [ ]:
# model.save("AttendanceSystem.keras")
# model.save_weights("model_weights.h5")

importing trained model

In [ ]:
from tensorflow.keras.utils import custom_object_scope

with custom_object_scope({'Attendance': Attendance}):
    # custom_objects = {'Attendance': Attendance.from_config}
    # custom_objects={'Functional':tf.keras.models.Model}
    model=tf.keras.models.load_model("AttendanceSystem.keras")

### Manual Testing

In [ ]:
test_img=os.path.join("images","Zinedine_Zidane_0001.jpg")
# test_img=os.path.join("D:\downloads\IMG_20220113_172032 (1) (2022_07_10 07_51_22 UTC).jpg")

In [ ]:
test_img

In [ ]:
test_img=load_image(test_img)

In [ ]:
test_img=np.expand_dims(test_img,axis=0)

In [ ]:
# test_img

In [ ]:
feature1=model.predict(test_img)[1]
# feature1

In [ ]:
feature2=model.predict(np.expand_dims(load_image(os.path.join("images","Zydrunas_Ilgauskas_0001.jpg")),axis=0))[1]

In [ ]:
dist1=EuclideanDistance(feature1,feature2)

In [ ]:
dist2=EuclideanDistance(feature1,model.predict(np.expand_dims(load_image(os.path.join("images","Zinedine_Zidane_0004.jpg")),axis=0))[1])

In [ ]:
dist1,dist2

In [ ]:
dist2<dist1

In [ ]:
person_feature={}

In [ ]:
for peeps in ['Zinedine_Zidane_0002', 'Zydrunas_Ilgauskas_0001', 'Zoe_Ball_0001', 'Zico_0001', 'Yuri_Malenchenko_0002', 'Zoran_Djindjic_0003']:
    person_feature[peeps]=EuclideanDistance(feature1,model.predict(np.expand_dims(load_image(os.path.join("images",peeps+".jpg")),axis=0))[1])

In [ ]:
len(person_feature)

In [ ]:
min_distance=min(person_feature.values())

In [ ]:
min_distance

In [ ]:
person_feature.values()

In [ ]:
for key in list(person_feature.keys()):
    if person_feature[key]==min_distance:
        print(key)
        break

In [ ]:
person_feature.keys()

### Realtime Comparison

In [ ]:
def isFace(gray,frame):
    faces=face_cascade.detectMultiScale(gray,scaleFactor=1.3,minNeighbors=3)
    if len(faces)>0:
        return (True,faces) #returns coordinates of face detected
    else:
        return(False,faces)

In [ ]:
def NewFace():
    cap=cv2.VideoCapture(0)
    try:
        while True:
            ret,frame=cap.read()
            if ret:
                # cv2.rectangle(frame,(140,50),(500,400),(0,0,255),2)
                gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
                face,coord=isFace(gray,frame)
                # print(coord)
                if face:
                    for (x,y,w,h) in coord:
                        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,255),2)
                cv2.imshow("test",frame)
                key=cv2.waitKey(1) & 0xFF
                if face and key==ord('c'):
                    name=input("Enter name: ")
                    print("processing..")
                    imgpath=os.path.join("user_imgs",name+".jpg")
                    #frame[y:y+h,x:x+w]
                    cv2.imwrite(imgpath,frame[coord[0][1]:coord[0][1]+coord[0][3],coord[0][0]:coord[0][0]+coord[0][2]]) #saves only the face and not surroundings
                    frame=np.expand_dims(load_image(imgpath),axis=0)
                    features=model.predict(frame)
                    print(features[0])
                    np.save("features/"+name+".npy",features[1])
                    print("done")
                    cap.release()
                    cv2.destroyAllWindows()
                    return
                elif key==ord('q'):
                    break
    except Exception as e:
        print(e,frame.shape)
    finally:
        cap.release()
        cv2.destroyAllWindows()

In [ ]:
NewFace()

In [ ]:
def SameFaceCheck():
    cap=cv2.VideoCapture(0)
    try:
        while True:
            ret,frame=cap.read()
            if ret:
                gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
                face,coord=isFace(gray,frame)
                if face:
                    for (x,y,w,h) in coord:
                        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,255),2)
                cv2.imshow("test",frame)
                key=cv2.waitKey(1) & 0xFF
                if face and key==ord('a'):
                    print("processing..")
                    imgpath=os.path.join("user_imgs","temp"+".jpg")
                    #frame[y:y+h,x:x+w]
                    cv2.imwrite(imgpath,frame[coord[0][1]:coord[0][1]+coord[0][3],coord[0][0]:coord[0][0]+coord[0][2]]) #saves only the face and not surroundings
                    img=np.expand_dims(load_image(imgpath),axis=0)
                    new=model.predict(img)[1]
                    person_feature={}
                    #traverses through all the features saved 
                    for feature in os.listdir("features"):
                        present=np.load("features/"+feature)
                        distance=EuclideanDistance(present,new)
                        person_feature[distance]=feature
                    min_dist=min(person_feature.keys())
                    print(person_feature[min_dist][:-4])
                    os.remove(os.path.join("user_imgs","temp.jpg"))
                    cap.release()
                    cv2.destroyAllWindows()
                    return
                elif key==ord('q'):
                    break
    except Exception as e:
        print(e)
    finally:
        cap.release()
        cv2.destroyAllWindows()

In [ ]:
SameFaceCheck()